# Transformers, what can they do?
Adopted from: https://huggingface.co/learn/nlp-course/chapter1/1

In this section, we will look at what Transformer models can do and use our first tool from the Huggingface Transformers library: the pipeline() function.


Transformers are everywhere!

Transformer models are used to solve all kinds of NLP tasks. Here are some of the companies and organizations using Hugging Face and Transformer models, who also contribute back to the community by sharing their models:

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.
- Text Classification (sentiment analysis)
- Zero-Shot Classification
- Text Generation
- Fill-Mask
- Token Classification (Named Entity Recognition, "ner")
- Question Answering
- Summarization

In [2]:
#%pip install datasets evaluate transformers[sentencepiece]

In [13]:
# how to see details about loaded model
# (pipeline_var).model.config

In [1]:
%pip show transformers

Name: transformers
Version: 4.31.0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /opt/conda/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: autogluon.multimodal
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import pipeline

2024-04-12 13:47:23.096164: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# # https://huggingface.co/docs/huggingface_hub/v0.22.2/en/package_reference/cache#huggingface_hub.HFCacheInfo.delete_revisions
# from huggingface_hub import scan_cache_dir
# cache_info = scan_cache_dir()
# delete_strategy = cache_info.delete_revisions()
# delete_strategy.execute()

# #     "81fd1d6e7847c99f5862c9fb81387956d99ec7aa"

# # )

# # print(f"Will free {delete_strategy.expected_freed_size_str}.")
# # Will free 7.9K.

#delete_strategy.execute()
def purgeHF():
    # purge cache
    !rm -rf ~/.cache/huggingface
purgeHF()

In [3]:
#from transformers import pipeline

sa = pipeline("sentiment-analysis",model = "distilbert-base-uncased-finetuned-sst-2-english")
#classifier("I've been waiting for a HuggingFace course my whole life.")

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [4]:
sa("I've been waiting for a HuggingFace course my whole life.")

[{'label': 'POSITIVE', 'score': 0.9598050713539124}]

In [5]:
sa(
    ["I've been waiting for a HuggingFace course my whole life.", "I hate this so much!"]
)

[{'label': 'POSITIVE', 'score': 0.9598050713539124},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [6]:
#from transformers import pipeline

zsc = pipeline("zero-shot-classification", model ="facebook/bart-large-mnli")
# classifier(
#     "This is a course about the Transformers library",
#     candidate_labels=["education", "politics", "business"],
# )

In [7]:
zsc(
    "This is a course about the Transformers library",
    candidate_labels=["education", "politics", "business"],
)

{'sequence': 'This is a course about the Transformers library',
 'labels': ['education', 'business', 'politics'],
 'scores': [0.8445995450019836, 0.11197379976511002, 0.04342666268348694]}

In [6]:
#purgeHF()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [3]:
#from transformers import pipeline

generator = pipeline("text-generation", model = "openai-community/gpt2")
#generator("In this course, we will teach you how to")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [5]:
generator("In this course, we will teach you how to", max_new_tokens = 100)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to use your camera to capture the scene from the front side of your car, and how to use your camera if you want to record in the full view view. We will give you a full-screen camera with the front camera recording of your view and then use that camera to create an animated short film that will show you the location of the car in front of you. You will also experience more than one camera in the film and will show people the video footage of your car in the video room.'}]

In [5]:
#from transformers import pipeline

#generator = pipeline("text-generation", model="distilgpt2")
generator(
    "In this course, we will teach you how to",
    max_length=30,
    num_return_sequences=2,
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to control your phone—and I will give you how. Your only goal will be to understand what it'},
 {'generated_text': 'In this course, we will teach you how to create simple things without having to deal with your own problems."\n\n\n\nPlease sign up for'}]

In [6]:
#from transformers import pipeline

unmasker = pipeline("fill-mask", model = "google-bert/bert-base-uncased")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [10]:
unmasker("This course will teach you all about [MASK] models.", top_k=5)
#unmasker("Hello I'm a [MASK] model.")

[{'score': 0.3366125822067261,
  'token': 2535,
  'token_str': 'role',
  'sequence': 'this course will teach you all about role models.'},
 {'score': 0.08371713012456894,
  'token': 2449,
  'token_str': 'business',
  'sequence': 'this course will teach you all about business models.'},
 {'score': 0.05718283727765083,
  'token': 1996,
  'token_str': 'the',
  'sequence': 'this course will teach you all about the models.'},
 {'score': 0.04392540082335472,
  'token': 2115,
  'token_str': 'your',
  'sequence': 'this course will teach you all about your models.'},
 {'score': 0.02668466977775097,
  'token': 4827,
  'token_str': 'fashion',
  'sequence': 'this course will teach you all about fashion models.'}]

In [2]:
#from transformers import pipeline

#ner = pipeline("ner", grouped_entities=True)
ner = pipeline("ner",model = "dbmdz/bert-large-cased-finetuned-conll03-english")


2024-04-12 14:00:18.405713: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSe

tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

In [7]:
ner("My name is Kurt and I work in at Cal Poly in San Luis Obispo.",aggregation_strategy="simple")

[{'entity_group': 'PER',
  'score': 0.99922895,
  'word': 'Kurt',
  'start': 11,
  'end': 15},
 {'entity_group': 'ORG',
  'score': 0.9969755,
  'word': 'Cal Poly',
  'start': 33,
  'end': 41},
 {'entity_group': 'LOC',
  'score': 0.9872705,
  'word': 'San Luis Obispo',
  'start': 45,
  'end': 60}]

In [8]:
#from transformers import pipeline

question_answerer = pipeline("question-answering",model="distilbert/distilbert-base-cased-distilled-squad")



config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [11]:
question_answerer(
    question="Where do I work?",
    context="My name is Kurt and I work in at Cal Poly in San Luis Obispo.",
)

{'score': 0.7860171794891357, 'start': 33, 'end': 41, 'answer': 'Cal Poly'}

In [14]:
#question_answerer.model.config

In [4]:

# This one doesn't run
#from transformers import pipeline

#summarizer = pipeline("summarization")
#summarizer = pipeline("summarization", model = "facebook/bart-large-cnn")

summarizer(
    """
    America has changed dramatically during recent years. Not only has the number of 
    graduates in traditional engineering disciplines such as mechanical, civil, 
    electrical, chemical, and aeronautical engineering declined, but in most of 
    the premier American universities engineering curricula now concentrate on 
    and encourage largely the study of engineering science. As a result, there 
    are declining offerings in engineering subjects dealing with infrastructure, 
    the environment, and related issues, and greater concentration on high 
    technology subjects, largely supporting increasingly complex scientific 
    developments. While the latter is important, it should not be at the expense 
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other 
    industrial countries in Europe and Asia, continue to encourage and advance 
    the teaching of engineering. Both China and India, respectively, graduate 
    six and eight times as many traditional engineers as does the United States. 
    Other industrial countries at minimum maintain their output, while America 
    suffers an increasingly serious decline in the number of engineering graduates 
    and a lack of well-educated engineers.
"""
)

[{'summary_text': 'America has changed dramatically during recent years. The number of graduates in traditional engineering disciplines has declined. There are declining offerings in engineering subjects dealing with infrastructure, the environment, and related issues. Rapidly developing economies such as China and India continue to encourage and advance the teaching of engineering.'}]